In [ ]:
import numpy as np
import tncontract as tn
import qutip as qt
import time
from loguru import logger

from itertools import product
from src.simulation_utils import *

In [ ]:
n_sites = 20 # number of sites
psi = tn.onedim.init_mps_allzero(n_sites, 2) # initial state all zeros
cct_depth = 100

# limit the bond dimension
chi = int(2 * np.log2(n_sites))
# chi=None

t = time.time()

for _ in range(int(cct_depth/3)):
    # two qubit layer
    cct_layer_1, cct_layer_2 = random_two_qubit_gate_ladder(n_sites) 
    psi = tn.onedim.contract_mps_mpo(psi,cct_layer_1)
    psi.left_canonise(chi=chi)
    psi = tn.onedim.contract_mps_mpo(psi,cct_layer_2)
    psi.left_canonise(chi=chi)
    
    # single qubit layer
    cct_layer = random_single_qubit_gate_layer(n_sites)
    psi = tn.onedim.contract_mps_mpo(psi,cct_layer)
    psi.left_canonise(chi=chi)

psi.left_canonise(normalise=True)
logger.info("Took {:.2f}s to run {} circuits on {} qubits".format(time.time()-t, cct_depth, n_sites))

In [ ]:
for s in psi:
    logger.info(s.shape)

In [ ]:
psi_vec = tn.onedim.contract_virtual_indices(psi)

In [ ]:
psi_vec.fuse_indices('physout','physout')

In [ ]:
psi_vec.shape

In [ ]:
qt.Qobj(psi_vec.data.reshape(-1,1))